In [1]:
import dask.array as ds
import dask
import os
from satpy import Scene, MultiScene, find_files_and_readers
from satpy.composites import GenericCompositor, DifferenceCompositor
from satpy.writers import to_image
from trollimage.colormap import spectral
from pyresample.geometry import AreaDefinition
import xarray as xr
import yaml
import cartopy
from glob import glob

ERROR 1: PROJ: proj_create_from_database: Open of /home/janky/miniforge3/envs/ALGOTLvisTest2/share/proj failed


In [2]:
kvarnakershamn_test = AreaDefinition.from_area_of_interest(
    area_id="kvarnakershamn", 
    projection='epsg:3006', 
    shape=[2000, 2000], 
    center=[686119.299, 6345703.075], 
    resolution=[10, 10],
    )

In [3]:
base_path = '/home/janky/Data/ALGOTL/'
dlist = [os.path.join(base_path, dir) for dir in os.listdir(base_path)]

In [4]:
scn_list = []

files = find_files_and_readers(base_dir=base_path, reader='msi_safe')
scn = Scene(filenames=files, reader='msi_safe')
scn_list.append(scn)
scn_list

In [5]:
mscn = MultiScene(scn_list)
mscn.load(['B03', 'B04'])
algotl_mscn = mscn.resample(kvarnakershamn_test)
algotl_scn = algotl_mscn.blend()

In [6]:
algotl_scn['kahru2'] = algotl_scn['B03'] / algotl_scn['B04']

In [17]:
mins = .8
maxs = 1

img = to_image(algotl_scn['kahru2'])
img.stretch('crude', min_stretch=[mins], max_stretch=[maxs])
spectral.set_range(mins, maxs)
# spectral.set_range(1, 2)
img.colorize(spectral)
img.show()

In [7]:
import geopandas as gpd

In [8]:
sub_basin_shp = '/home/janky/Data/Shapefiler/Sub-basins_Baltic_Sea/Havsomr_SVAR_2016_3b.shp'

In [9]:
def basin_geodataframe():
    """Method to recreate the GeoDataFrame containing the Baltic Sea
    sub-basins according to SVAR_2016.
    """
    basin_mapping_SVAR = {
        1: 'Bottenviken',
        2: 'Norra Kvarken',
        3: 'Bottenhavet',
        4: 'Ålands hav',
        5: 'Skärgårdshavet',
        6: 'Finska viken',
        7: 'Norra Gotlandshavet',
        8: 'Västra Gotlandshavet',
        9: 'Östra Gotlandshavet',
        10: 'Rigabukten',
        11: 'Gdanskbukten',
        12: 'Bornholmshavet och Hanöbukten',
        13: 'Arkonahavet och Södra Öresund',
        14: 'Bälthavet',
        15: 'Öresund',
        16: 'Kattegatt',
        17: 'Skagerrak',
    }
    
    # TODO Rename sub_basin_shp to be easily identified
    # TODO add file as argument
    gdf_basin = gpd.read_file(sub_basin_shp)
    basin_geometries = gdf_basin[['BASIN_NR', 'geometry']]
    basin_data = basin_geometries.dissolve(
        by='BASIN_NR', as_index=True)
    basin_data['basin_name'] = [
        basin_mapping_SVAR[item] for item in basin_data.index]
    return basin_data

In [10]:
basin_gdf = basin_geodataframe()

In [57]:
basin_gdf.crs

<Projected CRS: EPSG:3006>
Name: SWEREF99 TM
Axis Info [cartesian]:
- N[north]: Northing (metre)
- E[east]: Easting (metre)
Area of Use:
- name: Sweden - onshore and offshore.
- bounds: (10.03, 54.96, 24.17, 69.07)
Coordinate Operation:
- name: SWEREF99 TM
- method: Transverse Mercator
Datum: SWEREF99
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [11]:
basin_dataarray = basin_gdf.geometry.to_xarray()

In [12]:
masked_scn = algotl_scn.load(['kahru2'])

In [14]:
masked_scn['masked_kahru2'] = masked_scn['kahru2'].where(xr.apply_ufunc(lambda x, y: x.intersects(y), masked_scn['kahru2'], basin_dataarray))

TypeError: 'NoneType' object is not subscriptable

In [62]:
algotl_scn['masked_kahru2'] = algotl_scn['kahru2'].where(algotl_scn['kahru2'].intersects(basin_gdf))

AttributeError: 'DataArray' object has no attribute 'intersects'

In [61]:
algotl_scn['kahru2']

<xarray.DataArray (y: 2000, x: 2000)> Size: 16MB
dask.array<truediv, shape=(2000, 2000), dtype=float32, chunksize=(2000, 2000), chunktype=numpy.ndarray>
Coordinates:
    band         int64 8B 1
    spatial_ref  int64 8B 0
    crs          object 8B PROJCRS["SWEREF99 TM",BASEGEOGCRS["SWEREF99",DATUM...
  * y            (y) float64 16kB 6.356e+06 6.356e+06 ... 6.336e+06 6.336e+06
  * x            (x) float64 16kB 6.761e+05 6.761e+05 ... 6.961e+05 6.961e+05
Attributes:
    name:       kahru2
    _satpy_id:  DataID(name='kahru2')

In [63]:
algotl_scn['masked_kahru2']

<xarray.DataArray (y: 2000, x: 2000)> Size: 16MB
dask.array<broadcast_to, shape=(2000, 2000), dtype=float32, chunksize=(2000, 2000), chunktype=numpy.ndarray>
Coordinates:
    band         int64 8B 1
    spatial_ref  int64 8B 0
    crs          object 8B PROJCRS["SWEREF99 TM",BASEGEOGCRS["SWEREF99",DATUM...
  * y            (y) float64 16kB 6.356e+06 6.356e+06 ... 6.336e+06 6.336e+06
  * x            (x) float64 16kB 6.761e+05 6.761e+05 ... 6.961e+05 6.961e+05
Attributes:
    name:       masked_kahru2
    _satpy_id:  DataID(name='masked_kahru2')